<a href="https://colab.research.google.com/github/thisispriyanshugupta/Webpage-Question-Answering/blob/main/Webpage_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from bs4 import BeautifulSoup
import requests  # Import requests module

from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast #GPT2Tokenizer, GPT2LMHeadModel

In [4]:

def get_answer(url, question):
    # Fetch the webpage content
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    text = ' '.join(p.get_text() for p in soup.find_all('p'))

    # Load pre-trained DistilBERT model and tokenizer
    model_name = "distilbert-base-uncased"
    model = DistilBertForQuestionAnswering.from_pretrained(model_name)
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

    # Tokenize the input
    inputs = tokenizer(text, question, return_tensors="pt", padding=True, truncation=True)

    # Get the answer
    start_scores, end_scores = model(**inputs).start_logits, model(**inputs).end_logits
    start_idx = start_scores.argmax()
    end_idx = end_scores.argmax()
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx+1]))

    # If no answer is generated, return a default response
    if not answer:
        return "I don’t know the answer"

    return answer

# Sample input
input_data = {
    "url": "https://en.wikipedia.org/wiki/Generative_artificial_intelligence",
    "question": "What are the concerns around Generative AI?"
}

# Get the answer
output = get_answer(input_data['url'], input_data['question'])
print({"answer": output})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'answer': 'a wide range of industries, including software development, healthcare, finance, entertainment, customer service, [ 13 ] sales and marketing, [ 14 ] art, writing, [ 15 ] fashion, [ 16 ] and product design. [ 17 ] however, concerns have been raised about the potential misuse of generative ai such as cybercrime, the use of fake news or deepfakes to decei'}
